In [31]:
# %%
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import pandas as pd
import seaborn as sns
import math
from collections import Counter
from functools import reduce
#For interactive plots 

import plotly.graph_objects as go #pip install plotly
import dash
from dash import dcc, html #pip install dash
from dash.dependencies import Input, Output #pip install dash-bootstrap-components

#load the statistical libraries
from statsmodels.stats import diagnostic
from scipy import stats

from src.data.utils import get_franchise_movies, get_franchise_data, get_inflation_rate, get_movie, get_genre_counts


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
#loading the inflation rate data set from the columbia bank 
inflation_rate_df=get_inflation_rate()

#loading the complete data set 
movies_df_raw = pd.read_csv('data/movie_metadata_with_tmdb.csv')

#loading the data set for the franchise movies 
franchise_df=get_franchise_movies(movies_df_raw,inflation_rate_df,'data/missing_dates_manualsearch.csv')

#Data frame with all the movies with processing
movies_df=get_movie(movies_df_raw,inflation_rate_df)

#Data frame with movies that are not part of a franchise
movies_no_franchise_df = movies_df[movies_df['collection_id'].isna()].reset_index(drop=True)

#Data frame of only the franchise name and their characteristics
franchise_data_df = get_franchise_data(franchise_df)

# Drop franchises where any movie in the franchise does not have a release date
franchise_df = franchise_df.groupby('collection_id').filter(lambda x: x['Movie release date'].notna().all())
#print("Number of movies with all movies having release dates : ", len(franchise_df_2))


In [33]:
from sklearn.feature_extraction.text import CountVectorizer
def get_1_2_movies(data): 
    """
    Function to get the 1_st and 2_nd movie franhcise to analyse it in the tree and KNN model
    """
    # Drop unnecessary columns
    data.drop(columns=[
    'Freebase movie ID', 'Movie release date', 'Movie languages (Freebase ID:name tuples)', 
    'vote_count', 'tmdb_original_language', 'Movie release date corrected', 'box_office', 
    'Movie runtime', 'Movie countries (Freebase ID:name tuples)', 'CPI', 'budget', 'tmdb_id','profit'
    ], inplace=True)
    
    data['movie_order'] = data['movie_order'].astype(int)

    data = data[(data['movie_order'] == 1) | (data['movie_order'] == 2)]
    data = data.pivot(index='collection_id', columns='movie_order')
    data.columns = ['_'.join(map(str, col)).strip() for col in data.columns]
    data = data.drop(columns =['years_diff_bt_pre_movies_1','collection_size_2'], axis=1)

    #movie_genre = data['genres'].apply(lambda x: ', '.join(set([genre for sublist in x for genre in sublist])))

    return data 
    
    

In [34]:
franchise_tree = get_1_2_movies(franchise_df)

In [42]:

franchise_tree=franchise_tree.dropna()
franchise_tree.head()


,Wikipedia movie ID_1,Wikipedia movie ID_2,Movie name_1,Movie name_2,Movie genres (Freebase ID:name tuples)_1,Movie genres (Freebase ID:name tuples)_2,collection_name_1,collection_name_2,vote_average_1,vote_average_2,...,real_revenue_2,real_budget_1,real_budget_2,real_profit_1,real_profit_2,ratio_revenue_budget_1,ratio_revenue_budget_2,num_genres_1,num_genres_2,collection_size_1
collection_id,,,,,,,,,,,,,,,,,,,,,
10.0,52549.0,53964.0,Star Wars Episode IV: A New Hope,Star Wars Episode V: The Empire Strikes Back,"{""/m/06n90"": ""Science Fiction"", ""/m/03k9fj"": ""...","{""/m/06n90"": ""Science Fiction"", ""/m/03k9fj"": ""...",Star Wars Collection,Star Wars Collection,8.204,8.392,...,1.206333e+09,2.464762e+07,4.033246e+07,764398007.0,520375067.0,70.490728,29.909726,3.0,3.0,6.0
84.0,54166.0,81503.0,Indiana Jones and the Raiders of the Lost Ark,Indiana Jones and the Temple of Doom,"{""/m/03k9fj"": ""Adventure"", ""/m/0hj3m_6"": ""Cost...","{""/m/03btsm8"": ""Action/Adventure"", ""/m/0hj3m_6...",Indiana Jones Collection,Indiana Jones Collection,7.928,7.300,...,7.463909e+08,4.033246e+07,6.273939e+07,371925971.0,305107271.0,21.662554,11.896688,2.0,2.0,4.0
119.0,173941.0,173944.0,The Lord of the Rings: The Fellowship of the Ring,The Lord of the Rings: The Two Towers,"{""/m/0hj3n2k"": ""Fantasy Adventure"", ""/m/03k9fj...","{""/m/0hj3n2k"": ""Fantasy Adventure"", ""/m/03k9fj...",The Lord of the Rings Collection,The Lord of the Rings Collection,8.417,8.401,...,2.074987e+09,2.083844e+08,1.770147e+08,778530324.0,847047111.0,9.371294,11.722115,3.0,3.0,3.0
151.0,277006.0,27098.0,Star Trek: The Motion Picture,Star Trek II: The Wrath of Khan,"{""/m/06n90"": ""Science Fiction"", ""/m/0hqxf"": ""F...","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",Star Trek: The Original Series Collection,Star Trek: The Original Series Collection,6.500,7.443,...,2.173472e+08,1.008312e+08,2.688831e+07,94000000.0,85000000.0,3.088889,8.083333,3.0,4.0,6.0
230.0,2466773.0,73875.0,The Godfather,The Godfather Part II,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/0gw5w78"": ""G...","{""/m/0lsxr"": ""Crime Fiction"", ""/m/0gw5w78"": ""G...",The Godfather Collection,The Godfather Collection,8.690,8.600,...,4.324536e+08,1.344415e+07,2.912900e+07,262500000.0,180000000.0,44.750000,14.846154,2.0,2.0,3.0


In [49]:
X = franchise_tree[['vote_average_1', 'years_diff_bt_pre_movies_2', 'collection_size_1','run_time_1', 'run_time_2', 'ratio_revenue_budget_1', 'num_genres_1', 'num_genres_2', 'ratio_revenue_budget_2', 'vote_average_1']]
Y = franchise_tree['ratio_revenue_budget_2'] # is it possible to do it with 2 features --> vote_2


In [50]:
# %%
import sklearn
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [51]:
# Define the bins and labels
bins = [-np.inf, -1, 0, 1, 2, 5, 10, np.inf]
labels = ['Very Low', 'Low', 'Below Average', 'Average', 'Above Average', 'High', 'Very High']

# Transform Y into non-continuous values
Y_binned = pd.cut(y_train, bins=bins, labels=labels)

# Print the transformed Y
print(Y_binned)
print(y_train)

collection_id
91361.0         Very High
167613.0             High
43072.0     Below Average
86115.0     Above Average
9518.0      Above Average
                ...      
51509.0              High
735.0       Above Average
32135.0     Below Average
104830.0          Average
43563.0              High
Name: ratio_revenue_budget_2, Length: 200, dtype: category
Categories (7, object): ['Very Low' < 'Low' < 'Below Average' < 'Average' < 'Above Average' < 'High' < 'Very High']
collection_id
91361.0     10.213527
167613.0     5.908533
43072.0      0.685150
86115.0      2.673366
9518.0       2.661489
              ...    
51509.0      6.458037
735.0        2.870556
32135.0      0.317186
104830.0     1.698746
43563.0      5.438848
Name: ratio_revenue_budget_2, Length: 200, dtype: float64


In [52]:
# %%
from sklearn.tree import DecisionTreeClassifier

decision_tree_classifier = DecisionTreeClassifier(random_state=42)

decision_tree_classifier.fit(X_train, Y_binned)

y_test_pred_rf = decision_tree_classifier.predict(X_test)

#acc = accuracy_fn(y_test, y_test_pred_rf)
depth = decision_tree_classifier.get_depth()
#print(f"Accuracy: {acc}, Depth: {depth}")   

# Visualize the decision tree
from sklearn.tree import plot_tree
plt.figure(figsize=(100, 100))
plot_tree(decision_tree_classifier, filled=True)
plt.show()